# NLP (Natural Language Processing)

NLP son las siglas de Procesamiento del Lenguaje Natural (Natural Language Processing en inglés). Es una rama de la inteligencia artificial que se centra en la interacción entre las computadoras y el lenguaje humano. En otras palabras, NLP es una forma para que las computadoras entiendan, interpreten y manipulen el lenguaje humano.

El NLP combina la ciencia de la computación y la lingüística, por lo que las computadoras pueden interactuar con los textos humanos de una manera útil. Por ejemplo, el NLP se utiliza en los correctores ortográficos, en los sistemas de respuesta a preguntas (como los asistentes virtuales), en los traductores automáticos, en los sistemas de análisis de sentimientos, entre otros.

NLTK, que significa Natural Language Toolkit, es una biblioteca líder en Python para el procesamiento del lenguaje natural (NLP). Fue desarrollada por investigadores y estudiantes en el campo de la lingüística y la ciencia de la computación para apoyar la enseñanza y el trabajo en NLP y en la lingüística computacional.

NLTK proporciona interfaces fáciles de usar a más de 50 cuerpos y recursos léxicos, como WordNet, junto con un conjunto de bibliotecas de procesamiento de texto para la clasificación, tokenización, derivación, etiquetado, análisis y razonamiento semántico.

Algunos de los usos comunes de NLTK incluyen:

- Tokenización: Divide el texto en frases o palabras.
- Stemming y Lemmatization: Reduce las palabras a su raíz.
- Etiquetado de partes del discurso (POS tagging): Identifica la parte de la oración de una palabra.
- Análisis de sentimientos: Identifica la actitud del hablante.
- Clasificación de texto: Clasifica el texto en diferentes categorías.
- Extracción de información: Extrae información estructurada de texto no estructurado.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_dark"
# text mining
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud

from langdetect import detect
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Do not limit the width of columns
pd.set_option('display.max_colwidth', None)  # Show the entire content of each cell


In [ ]:
df_reviews = pd.read_csv('http://data.insideairbnb.com/portugal/norte/porto/2022-12-16/data/reviews.csv.gz', parse_dates=['date'])
#df_reviews.to_csv('input/reviews.csv', index=False)

In [ ]:
df_listing = pd.read_csv('http://data.insideairbnb.com/portugal/norte/porto/2022-12-16/visualisations/listings.csv', index_col= ["id"])
#df_listing.to_csv('output/listings.csv', index=False)

In [ ]:
df_oporto_review = df_listing # Copio el df aquí para usar las review después en la sección de la Nube de palabras

In [ ]:
df_reviews = pd.merge(df_reviews, df_oporto_review[['host_id', 'host_name', 'name']], left_on = "listing_id", right_index=True, how = "left")
df_reviews = df_reviews.reset_index(drop=True)
df_reviews = df_reviews[['name', 'host_id', 'host_name', 'date', 'reviewer_id', 'reviewer_name', 'comments']]
df_reviews

In [ ]:
# Definimos una función que detecta el idioma de un texto (cuidado que tarda ~14min en ejecutarse)
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'

# La aplicamos a la columna de comentarios
df_reviews['language'] = df_reviews['comments'].apply(detect_language)


# Guardo este notebook porque tarda 14 minutos en detectar los idiomas de los comentarios.
df_reviews.to_csv('output/df_reviews.csv')

In [ ]:
# Cargo el archivo con los idiomas detectados
df_reviews = pd.read_csv('output/df_reviews.csv', low_memory=False)
df_reviews

In [ ]:
# Graficamos con plotly la columna de language para ver cuántos comentarios hay en cada idioma
fig = px.histogram(df_reviews, x='language')
fig.update_layout(title_text='Número de comentarios por idioma')
fig

In [ ]:
import ssl
# The purpose of this code is to handle SSL certificate verification when making HTTPS requests. By default, Python's ssl module verifies the SSL certificates of the websites it connects to. However, in some cases, such as when working with self-signed certificates or in development environments, it may be necessary to disable certificate verification.
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')
nltk.download('vader_lexicon')

In [ ]:
# Create a copy of the DataFrame to avoid warnings
df_reviews = df_reviews.copy()

# take out empty comments (530)
df_reviews = df_reviews[df_reviews['comments'].notnull()]

# remove non-alphanumeric characters and make everything lowercase
df_reviews.loc[:, 'comments'] = df_reviews['comments'].str.replace('[^a-zA-Z0-9\sáéíóúÁÉÍÓÚñÑçÇ~]+', '').str.lower()

# remove windows new line
df_reviews.loc[:, 'comments'] = df_reviews['comments'].str.replace('\r\n', "")

# remove stopwords (from nltk library)
stop_words = []
for language in ["english", "spanish", "french", "portuguese", "german", "italian"]:
    stop_words.extend(stopwords.words(language))
df_reviews.loc[:, 'comments'] = df_reviews['comments'].apply(lambda x: " ".join([i for i in x.split() if i not in stop_words]))

# remove punctuation and replace x spaces by one space
df_reviews.loc[:, 'comments'] = df_reviews['comments'].str.replace('[^\w\s]', " ").str.replace('\s+', ' ')

df_reviews.comments.values[2]  # print same comments again

In [ ]:
texts = df_reviews.comments.tolist()

vec = CountVectorizer().fit(texts)
bag_of_words = vec.transform(texts)
sum_words = bag_of_words.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]

cvec_df = pd.DataFrame.from_records(words_freq, columns= ['words', 'counts']).sort_values(by="counts", ascending=False)
cvec_df.head(10)

In [ ]:
excluded_words = ['br', 'us', 'die', 'made']
cvec_df = pd.DataFrame.from_records(words_freq, columns=['words', 'counts']).sort_values(by="counts", ascending=False)

# Filtrar las palabras excluidas
cvec_df = cvec_df[~cvec_df['words'].isin(excluded_words)]

cvec_dict = dict(zip(cvec_df.words, cvec_df.counts))

wordcloud = WordCloud(width=800, height=400)
wordcloud.generate_from_frequencies(frequencies=cvec_dict)
plt.figure(figsize=(20, 10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.tight_layout(pad=0)
plt.savefig('img/wordcloud.png')
plt.show()